In [2]:
search_for = 127
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0039937496185302734
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 262209281
type: [1, 1, 1, 1, 127] 127
cases of this type: 260144641
100000 11.01479063341554
200000 19.1373119547428
300000 19.30273001480092
400000 18.97051455454547
500000 19.159476043473862
600000 19.710511974029412
700000 18.784058679132716
800000 18.886856035852837
900000 18.94837877749895
1000000 16.610753340027596
1100000 8.842000148842962
1200000 18.681743752884614
1300000 18.67033086838221
1400000 18.56462603375761
1500000 18.74130033355805
1600000 19.60453913360592
1700000 18.608800338457808
1800000 18.565790814819422
1900000 18.662375831802034
2000000 19.21889953055029
2100000 5.007328511442603
2200000 20.966265419143305
2300000 22.583652748412113
2400000 22.046440755754684
2500000 22.200630260010428
2600000 22.786629337348504
2700000 21.107433189956485
2800000 21.105707047221653
2900000 21.142931876774178
3000000 20.

29400000 16.587869634824308
29500000 16.949264051834593
29600000 16.061248008826826
29700000 11.58072453272971
29800000 11.350582741568081
29900000 16.185694945822007
30000000 16.80314992511817
30100000 16.55332603142189
30200000 18.19505776507761
30300000 17.38195715990993
30400000 18.202029813161058
30500000 11.77367353002132
30600000 14.203310554956657
30700000 18.20442701338869
30800000 17.867771846777593
30900000 17.615279922018004
31000000 9.4185168647353
31100000 15.650647642133832
31200000 17.80520203152313
31300000 17.428091163614504
31400000 17.400928099813477
31500000 16.312492361010655
31600000 16.04050866634994
31700000 16.206949371779647
31800000 5.3698590923329
31900000 16.62638191637737
32000000 15.705092609016155
32100000 16.174608409599408
32200000 16.668042036247545
32300000 17.48021413929227
32400000 17.462832715457516
32500000 15.5304454046672
32600000 10.281812528391875
32700000 17.70176610459956
32800000 17.591394455480707
32900000 17.77974282078057
33000000 18.0

59200000 15.823753208759557
59300000 15.749572132701001
59400000 15.925506008753517
59500000 15.922226882709765
59600000 15.262716370448482
59700000 15.468579965099021
59800000 14.79521962479043
59900000 8.48389051002372
60000000 12.75658549872109
60100000 13.384974424921134
60200000 13.972833763946225
60300000 13.445886751713438
60400000 12.53876551639354
60500000 6.711506474783367
60600000 13.54317012706544
60700000 13.49587700802927
60800000 13.292026936022436
60900000 13.29949648221869
61000000 8.203511108838587
61100000 13.897031665576645
61200000 15.079736496835338
61300000 15.01379161247405
61400000 15.547338819573397
61500000 16.124963163890246
61600000 15.571650240631428
61700000 14.952621956782782
61800000 15.193387140704033
61900000 15.11424393677535
62000000 6.273415056537992
62100000 13.63620355588377
62200000 13.442456231182017
62300000 13.417719972931202
62400000 13.403703669907227
62500000 6.621247375675144
62600000 12.866658671466563
62700000 13.43130750181383
62800000

88800000 6.904754270147399
88900000 9.834059943526434
89000000 12.016735779884897
89100000 8.1563819946761
89200000 7.6754598281079796
89300000 11.397326211731336
89400000 10.926861983528381
89500000 6.864816549916844
89600000 11.695731662423944
89700000 11.449085515230545
89800000 11.914468501300753
89900000 12.388838772312551
90000000 12.838815742320984
90100000 12.56253310075402
90200000 12.714316639276843
90300000 12.813964532941636
90400000 12.511733485146383
90500000 12.522711050037199
90600000 11.8733038595581
90700000 11.570663625403306
90800000 11.611349146284686
90900000 4.818326189411981
91000000 11.504560752817154
91100000 11.563306140131589
91200000 3.855418722647149
91300000 11.729692633060386
91400000 11.547966093555576
91500000 5.49323744371618
91600000 12.01035028707925
91700000 11.394176969076684
91800000 11.348827709091465
91900000 12.148471240194482
92000000 12.511927858828091
92100000 13.488517922158241
92200000 12.534292909817198
92300000 12.367006356147371
924000

117900000 4.0585343349817125
118000000 8.705324096025697
118100000 9.288361455094703
118200000 9.346102322824974
118300000 9.604802903864984
118400000 9.499406034439374
118500000 9.543622538088705
118600000 9.726367797885842
118700000 9.892006739609645
118800000 10.439909091528165
118900000 10.596936416009381
119000000 9.627502354604903
119100000 9.50021364988265
119200000 9.152586297230538
119300000 9.214245861404383
119400000 9.268012056229564
119500000 9.431766725759378
119600000 8.867804066401996
119700000 9.492841713154645
119800000 3.160544259938036
119900000 1.7295611698347906
120000000 6.673922128905587
120100000 9.013085930545323
120200000 9.05727329055578
120300000 9.290395200595164
120400000 9.279485310011877
120500000 9.342569996773404
120600000 9.121726462345318
120700000 9.695955620599586
120800000 9.579160213496019
120900000 11.206234615508926
121000000 10.687420657451101
121100000 10.182454072553927
121200000 9.926797448895838
121300000 9.625837157634708
121400000 9.923

146700000 6.153883977915166
146800000 7.474161106891734
146900000 7.753639483873599
147000000 7.98152485536616
147100000 7.9108570338355335
147200000 7.886510255285293
147300000 7.975690365320266
147400000 8.510104266025996
147500000 8.538543151694592
147600000 8.892908613083092
147700000 8.201414301689839
147800000 7.882672135321244
147900000 7.862282852638138
148000000 7.918454945828212
148100000 7.812716642621505
148200000 7.691873005284968
148300000 7.392125220056386
148400000 3.693220660563755
148500000 6.170883762427129
148600000 5.2180302020371965
148700000 3.333392685133926
148800000 7.536663412598524
148900000 7.690991323829882
149000000 7.732274134726244
149100000 7.69572506585214
149200000 7.80498860626468
149300000 7.6827361022493035
149400000 8.196499248314659
149500000 8.519301908547916
149600000 8.623555711826565
149700000 8.437908831422988
149800000 7.797170770778227
149900000 7.681905083127361
150000000 7.668139799433047
150100000 7.666343157653362
150200000 7.44925004

176000000 6.671614751010721
176100000 6.567846799854175
176200000 6.637725417816182
176300000 6.6953868896816155
176400000 6.653885184096964
176500000 6.424015308739712
176600000 6.258346301022009
176700000 6.022515099319618
176800000 4.915880609833361
176900000 3.9901398608805305
177000000 5.99374494126842
177100000 6.0886325159324715
177200000 3.251733131230938
177300000 5.05430022437983
177400000 5.82191629354836
177500000 5.885510628773635
177600000 2.5731953727656576
177700000 5.94537365394414
177800000 5.955616130525787
177900000 6.507805020511421
178000000 6.198045022184569
178100000 6.466090524321958
178200000 6.527114955366089
178300000 6.470515937452398
178400000 6.371219594039388
178500000 6.241745338908983
178600000 6.299330001022633
178700000 5.958066726233256
178800000 5.768053126545732
178900000 2.8417306988158906
179000000 5.857211224215174
179100000 5.774549746160699
179200000 5.55764381869158
179300000 2.4443226705483396
179400000 5.675177121437234
179500000 6.0500705

205200000 4.601723739268691
205300000 2.12021012501371
205400000 4.277400669775621
205500000 4.226601761541913
205600000 4.01796918193913
205700000 4.050154771968466
205800000 3.981581435031076
205900000 1.459205241862539
206000000 4.201794533191555
206100000 3.9796657630336143
206200000 3.949642795388966
206300000 4.019793817218085
206400000 4.176553099840244
206500000 1.81274562599279
206600000 4.489295038862514
206700000 4.533703449124983
206800000 4.36003282354556
206900000 4.367761749307563
207000000 4.507956487487196
207100000 4.415776367797135
207200000 4.438545096339331
207300000 3.0293499564137236
207400000 3.188289491144475
207500000 4.079832509460836
207600000 3.987337651718889
207700000 3.748966854682826
207800000 3.8117452365540183
207900000 3.005753424057435
208000000 2.4171750907297263
208100000 3.8162767028869746
208200000 3.8252683812466968
208300000 3.9339643956103916
208400000 4.04589911956938
208500000 3.7337715723330045
208600000 2.2323605215836286
208700000 4.2449

234000000 2.2304045689641727
234100000 2.1459179454641593
234200000 2.1537031140508116
234300000 2.0598538291514212
234400000 2.0177110461270886
234500000 2.004357488971919
234600000 0.7106733411976165
234700000 1.9661741325702649
234800000 1.9932849946721694
234900000 2.0572752319133816
235000000 2.049002337525656
235100000 2.041279525363475
235200000 2.0993749595689337
235300000 2.089375074786031
235400000 0.8964517320342182
235500000 2.113674419255372
235600000 2.1180510909126005
235700000 2.0693770584088287
235800000 0.9228204455216572
235900000 2.0229836482405865
236000000 2.013077532614968
236100000 1.9384408202849785
236200000 1.902482555003595
236300000 1.9590607388506294
236400000 1.8419053745814595
236500000 1.7936376546956203
236600000 1.0365870284507495
236700000 1.4699697247140426
236800000 1.8088139735359736
236900000 1.825408168321403
237000000 1.967814866169876
237100000 1.9011188650900503
237200000 1.898565393539691
237300000 1.9082038478954348
237400000 1.757961509968

262000000 0.001212085259437561
262100000 0.0006236983339250089
best so far: 0
type: [1, 1, 127, 1, 1] 127
cases of this type: 16129
262200000 5.394423209561243e-05
best so far: 0
type: [1, 127, 1, 1, 1] 127
cases of this type: 127
best so far: 0
type: [127, 1, 1, 1, 1] 127
cases of this type: 1
best so far: 0
all cases: 262209281
best: 0
